# Deploying a web service to Azure Container Instance (ACI)
This notebook shows the steps for deploying a service: registering a model, creating an scoring script, creating the inference and deployment configs, and deploying model as a service to ACI.

In [32]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model

In [33]:
import azureml.core
print(azureml.core.VERSION)

1.0.57


# Get workspace
Load existing workspace from the config file info.

In [34]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

AMLServiceWS
AMLServiceRG
eastus
a4393d89-7e7f-4b0b-826e-72fc42c33d1f


# Register the model
Register an existing trained model, add description and tags.

In [35]:
#Register the model
from azureml.core.model import Model
model = Model.register(model_path = "original_model.pkl", # this points to a local file
                       model_name = "IBM-attrition-model", # this is the name the model is registered as
                       tags = {'area': "HR", 'type': "attrition"},
                       description = "Attrition model to understand attrition risk",
                       workspace = ws)

print(model.name, model.description, model.version)

Registering model IBM-attrition-model
IBM-attrition-model Attrition model to understand attrition risk 2


# Write the Entry Script
Write the script that will be used to predict on your model

In [36]:
%%writefile score.py
import pandas as pd
from sklearn.externals import joblib
from azureml.core.model import Model

def init():

    global original_model
    
    # Retrieve the path to the model file using the model name
    original_model_path = Model.get_model_path('IBM-attrition-model')
    
    original_model = joblib.load(original_model_path)
    
def run(raw_data):
    # Get predictions and explanations for each data point
    data = pd.read_json(raw_data)
    # Make prediction
    predictions = original_model.predict(data)
    return {'predictions': predictions.tolist()}

Overwriting score.py


# Create the InferenceConfig
Create the inference config that will be used when deploying the model

In [38]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='score.py', 
                                   extra_docker_file_steps='Dockerfile', 
                                   runtime='python',
                                   conda_file="myenv.yml"
                                   )


# Deploy Model as Webservice on Azure Container Instance

In [39]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "IBM_Attrition",  
                                                     "method" : "local_explanation"}, 
                                               description='Explain predictions on employee attrition')
                                               

service = Model.deploy(ws, name='predictattritionsvc7', models=[model], inference_config= inference_config, deployment_config=aciconfig)

service.wait_for_deployment(True)
print(service.state)

Creating image
Running......................................................
Succeeded
Image creation operation finished for image predictattritionsvc7:1, operation "Succeeded"
Creating service
Running........................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


In [40]:
print(service.scoring_uri)

http://e758561e-30dc-449c-b886-53dbdc0d5e35.eastus.azurecontainer.io/score


In [42]:
import json

sample = '{"Age":{"899":49},"BusinessTravel":{"899":"Travel_Rarely"},"DailyRate":{"899":1098},"Department":{"899":"Research & Development"},"DistanceFromHome":{"899":4},"Education":{"899":2},"EducationField":{"899":"Medical"},"EnvironmentSatisfaction":{"899":1},"Gender":{"899":"Male"},"HourlyRate":{"899":85},"JobInvolvement":{"899":2},"JobLevel":{"899":5},"JobRole":{"899":"Manager"},"JobSatisfaction":{"899":3},"MaritalStatus":{"899":"Married"},"MonthlyIncome":{"899":18711},"MonthlyRate":{"899":12124},"NumCompaniesWorked":{"899":2},"OverTime":{"899":"No"},"PercentSalaryHike":{"899":13},"PerformanceRating":{"899":3},"RelationshipSatisfaction":{"899":3},"StockOptionLevel":{"899":1},"TotalWorkingYears":{"899":23},"TrainingTimesLastYear":{"899":2},"WorkLifeBalance":{"899":4},"YearsAtCompany":{"899":1},"YearsInCurrentRole":{"899":0},"YearsSinceLastPromotion":{"899":0},"YearsWithCurrManager":{"899":0}}'

# sample = '{"Age":{"899":49},"BusinessTravel":{"899":"Travel_Rarely"},"DailyRate":{"899":1098},"Department":{"899":"Research & Development"},"DistanceFromHome":{"899":4},"Education":{"899":2},"EducationField":{"899":"Medical"},"EnvironmentSatisfaction":{"899":4},"Gender":{"899":"Female"},"HourlyRate":{"899":21},"JobInvolvement":{"899":3},"JobLevel":{"899":2},"JobRole":{"899":"Laboratory Technician"},"JobSatisfaction":{"899":3},"MaritalStatus":{"899":"Single"},"MonthlyIncome":{"899":711},"MonthlyRate":{"899":2124},"NumCompaniesWorked":{"899":7},"OverTime":{"899":"Yes"},"PercentSalaryHike":{"899":8},"PerformanceRating":{"899":4},"RelationshipSatisfaction":{"899":3},"StockOptionLevel":{"899":0},"TotalWorkingYears":{"899":2},"TrainingTimesLastYear":{"899":0},"WorkLifeBalance":{"899":3},"YearsAtCompany":{"899":1},"YearsInCurrentRole":{"899":0},"YearsSinceLastPromotion":{"899":0},"YearsWithCurrManager":{"899":0}}'


test_sample_encoded = bytes(sample,encoding = 'utf8')
prediction = service.run(input_data=test_sample_encoded)
print(prediction)

{'predictions': [0]}
